In [2]:
include("../../src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, Constraints, PSO, Objectives, Designs, Models, HitAndRun, TensorOps
using Base.Iterators
using LinearAlgebra
using NPZ
using HiGHS
using Polyhedra

# Mixture Constraints

In [40]:
N = 12
K = 3

a_2 = [0 -1 0]
b_2 = [-1/10]

a_3 = [0 0 1]
b_3 = [3/5]

a_factor_1 = [5 4 0]
b_factor_1 = [39/10]

a_factor_2 = [-20 5 0]
b_factor_2 = [-3]

A_og = Array{Float64}(vcat(a_2, a_3, a_factor_1, a_factor_2))
b_og = vcat(b_2, b_3, b_factor_1, b_factor_2)

A, b = reparameterize_simplex(A_og, b_og)

experiment = Experiments.create(N, K)
experiment = Experiments.with_linear_constraints(experiment, A, b)

Main.MetaDoE.Experiments.Experiment(Dict("1" => 1, "2" => 2, "3" => 3), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([0.0 -1.0; -1.0 -1.0; … ; -0.0 -1.0; 1.0 1.0], [0.2333333333333333, 0.26666666666666666, 0.8999999999999999, 2.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]), 12, 3)

In [36]:
A

7×2 Matrix{Float64}:
   0.0  -1.0
  -1.0  -1.0
   5.0   4.0
 -20.0   5.0
  -1.0  -0.0
  -0.0  -1.0
   1.0   1.0

In [41]:
verts = Constraints.get_vertices(experiment)

4×2 Matrix{Float64}:
 -0.0333333   0.266667
 -0.133333   -0.133333
  0.366667   -0.233333
 -0.0333333  -0.233333

In [23]:
model = Models.full_cubic
obj_crit = Objectives.D
objective = obj_crit ∘ model

runner_params = PSO.runner_params(200, 200, 1e-6)
pso_params = PSO.create_hyperparams(500)

context = PSO.create_context(
    experiment, 
    objective; 
    runner_params=runner_params,
    enforcer_type = ConstraintEnforcement.Parametric
)

runner_state, history = PSO.optimize(context)

Iteration: 0 Best score: 37.285513176991195
Iteration: 1 Best score: 37.175467992457065
Iteration: 2 Best score: 34.924092573741596
Iteration: 3 Best score: 34.924092573741596
Iteration: 4 Best score: 34.924092573741596
Iteration: 5 Best score: 34.924092573741596
Iteration: 6 Best score: 34.924092573741596
Iteration: 7 Best score: 34.924092573741596
Iteration: 8 Best score: 34.924092573741596
Iteration: 9 Best score: 34.924092573741596
Iteration: 10 Best score: 34.924092573741596
Iteration: 11 Best score: 34.924092573741596
Iteration: 12 Best score: 34.924092573741596
Iteration: 13 Best score: 34.924092573741596
Iteration: 14 Best score: 34.924092573741596
Iteration: 15 Best score: 34.924092573741596
Iteration: 16 Best score: 34.924092573741596
Iteration: 17 Best score: 34.924092573741596
Iteration: 18 Best score: 34.924092573741596
Iteration: 19 Best score: 34.924092573741596
Iteration: 20 Best score: 34.924092573741596
Iteration: 21 Best score: 34.924092573741596
Iteration: 22 Best s

(Main.MetaDoE.PSO.RunnerState(Main.MetaDoE.PSO.Swarm(Main.MetaDoE.PSO.ParticleState([0.08939381699234301 0.041075555960938065 … -0.01949899480038116 -0.004339275192639987; 0.13787456749583307 -0.0386144814058609 … 0.2666612914451108 -0.016026993526414082; … ; -0.05721678338229829 0.0997692171332587 … 0.07159756467491303 -0.02154457660441489; 0.3471272974958085 -0.14521936304916477 … 0.06721726437612073 -0.01041013919762882;;; -0.18869582049555209 -0.22647555962905644 … 0.16156764262577306 0.03838722993834409; -0.03982224191977812 -0.20885944900357942 … 0.015587126677367212 -0.0725424335170371; … ; -0.16414667263911903 -0.18665796161628548 … 0.1442560609284641 -0.10466963279072306; 0.015343483922181933 -0.10378276913980974 … 0.2357497335848952 -0.1875937417492202;;; -0.14891170534490095 -0.12846666788937278 … -0.020064627026916592 -0.09055370245882974; -0.06919018531586758 -0.14915797680314674 … 0.027416139374159362 -0.11392324106616908; … ; -0.14045448534047242 -0.09562958149434224 … 0

In [20]:
optimizer, optimum = PSO.get_optimizer(runner_state)

([-0.09028064795012106 -0.15269043495537965 -0.1476570276351669; -0.0033143177048279935 -0.2310475674040475 -0.1447872950362918; … ; 0.4699976532956094 -0.02207362618366942 -0.04792402711193991; 0.04617465466079553 0.33629462229241075 0.0022293416899757763], 33.651064586053096)